In [11]:
# 데이터 불러오기
from dotenv import load_dotenv
import pandas as pd
import os 
import numpy as np
from sqlalchemy import create_engine


# .env 파일에서 환경 부르기
load_dotenv()

# 환경 변수 사용
database = os.getenv("CON_STR")


# # csv 파일 경로
# csv_file_path = os.getenv("CSV_FILE_PATH")
# print(csv_file_path)

engine = create_engine(database)

In [7]:

# ALL Raw Data
sql1= ''' 
select * from cu.agg_CU_TEREA_Total_Sourcing
order by YYYYMM, id
'''

gg = pd.read_sql_query(sql=sql1, con=engine)



In [ ]:
gg.to_clipboard()

# CU 대상 TEREA/MIIX/FIIT/NEO Total Sourcing 피벗

In [3]:
# - flaXtar_ from 202302
sql2 = ''' 
select  
	t.YYYYMM,
	concat(FLAVORSEG_type3,' X ', New_TARSEGMENTAT) flavorXtar,
	count(distinct case when b.cigatype ='CC' then t.id end) CC,
	count(distinct case when b.cigatype ='HnB' then t.id end) HnB
from  cu.agg_CU_TEREA_Total_Sourcing t
	join cu.Fct_BGFR_PMI_Monthly a on t.id = a.id 
		and a.YYYYMM BETWEEN CONVERT(NVARCHAR(6), DATEADD(MONTH, -3, t.YYYYMM+'01'), 112)
				 	     AND CONVERT(NVARCHAR(6), DATEADD(MONTH, -1, t.YYYYMM+'01'), 112)	
	join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and b.CIGADEVICE =  'CIGARETTES' AND b.cigatype != 'CSV'  
	join cu.dim_Regional_area c on t.SIDO_nm = c.sido_nm
group BY 
	t.YYYYMM,
	concat(FLAVORSEG_type3,' X ', New_TARSEGMENTAT) 
'''

data2 = pd.read_sql_query(sql=sql2, con=engine)

pivot_flavor_tar = data2.pivot_table(index=['YYYYMM'], 
                                     columns='flavorXtar',
                                     values=['HnB', 'CC' ]
                                    ).sort_values([ 'YYYYMM'])
print(pivot_flavor_tar)



                 CC                                                        \
flavorXtar Fresh X  Fresh X 1MG Fresh X Below 1MG Fresh X LTS Fresh X ULT   
YYYYMM                                                                      
202302          0.0       140.0               NaN       156.0        21.0   
202303          0.0       272.0               NaN       338.0        57.0   
202304          0.0       286.0               NaN       300.0        64.0   
202305          0.0       276.0               NaN       326.0        66.0   
202306          0.0       259.0               NaN       292.0        61.0   
202307          0.0       241.0               NaN       319.0        54.0   
202308          0.0       247.0               NaN       247.0        53.0   
202309          0.0       253.0               NaN       313.0        51.0   
202310          0.0       279.0               NaN       335.0        58.0   
202311          0.0       251.0               NaN       309.0        47.0   

In [4]:
pivot_flavor_tar.to_clipboard()

In [5]:
# - Terea_user_past_type_M1
sql = ''' 
select  
	t.YYYYMM, 
	t.id,
	max(case when b.cigatype='HnB' and b.company = 'PMK' then 1 else 0 end) IQOS_Purchased,
	max(case when b.cigatype='CC' then 1 else 0 end) CC_Purchased,
	max(case when b.cigatype='HnB' and b.company != 'PMK' then 1 else 0 end) CompHnB_Purchased
from cu.agg_CU_TEREA_Total_Sourcing t
	join cu.Fct_BGFR_PMI_Monthly a on t.id = a.id 
		and a.YYYYMM BETWEEN CONVERT(NVARCHAR(6), DATEADD(MONTH, -3, t.YYYYMM+'01'), 112)
				 	     AND CONVERT(NVARCHAR(6), DATEADD(MONTH, -1, t.YYYYMM+'01'), 112)	
	join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and CIGADEVICE =  'CIGARETTES' AND  b.cigatype != 'CSV' 
	join cu.dim_Regional_area c on t.SIDO_nm = c.sido_nm
group BY 	    
	t.YYYYMM, 
	t.id
'''

data = pd.read_sql_query(sql=sql, con=engine)


# Past Type 
def categorize(row):
    categories = []
    if row['IQOS_Purchased'] == 1:
        categories.append('PMK HnB')
    if row['CC_Purchased'] == 1:
        categories.append('CC')
    if row['CompHnB_Purchased'] == 1:
        categories.append('Comp HnB')
    return ','.join(categories)

data['retype'] = data.apply(categorize, axis=1)
result = data.groupby(['YYYYMM', 'retype']).size().reset_index(name='N')
total = result['N'].sum()

pivot_past_type = result.pivot_table( 
                            index=['YYYYMM'],
                            columns='retype',
                            values='N'
                            ).sort_values(['YYYYMM'])
print(pivot_past_type)

pivot_past_type.to_clipboard()

retype      CC  CC,Comp HnB  Comp HnB  PMK HnB  PMK HnB,CC  \
YYYYMM                                                       
202302  2006.0        331.0     285.0   1312.0       623.0   
202303  4009.0        737.0     519.0   2113.0      1197.0   
202304  3786.0        815.0     438.0   1592.0      1157.0   
202305  4258.0        907.0     476.0   1530.0      1000.0   
202306  3724.0        751.0     424.0   1224.0       860.0   
202307  3645.0        731.0     356.0   1082.0       745.0   
202308  3418.0        659.0     374.0    930.0       659.0   
202309  3905.0        799.0     398.0    970.0       665.0   
202310  4129.0        743.0     379.0    975.0       694.0   
202311  3632.0        688.0     374.0    812.0       575.0   
202312  3699.0        703.0     322.0    780.0       550.0   
202401  3860.0        688.0     341.0    682.0       459.0   
202402  4012.0        729.0     377.0    648.0       434.0   
202403  4356.0        766.0     393.0    621.0       416.0   
202404  

In [6]:
# - Terea_user_Current_type_M1 (IQOS TEREA 전용)

sql3 = ''' 
with temp as (
select  
	t.YYYYMM, 
	t.id,
	max(case when b.cigatype='HnB' and b.company = 'PMK' then 1 else 0 end) IQOS_Purchased,
	max(case when b.cigatype='CC' then 1 else 0 end) CC_Purchased,
	max(case when b.cigatype='HnB' and b.company != 'PMK' then 1 else 0 end) CompHnB_Purchased
from  cu.agg_CU_TEREA_Total_Sourcing t
	join cu.Fct_BGFR_PMI_Monthly a on t.id = a.id 
		and a.YYYYMM = t.YYYYMM
	join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and CIGADEVICE =  'CIGARETTES' AND  b.cigatype != 'CSV' 
	join cu.dim_Regional_area c on t.SIDO_nm = c.sido_nm
group BY 	    	
	t.YYYYMM, 
	t.id
)
select YYYYMM,
    'IQOS' +
    CASE WHEN CompHnB_Purchased = 1 THEN ' + Comp. HnB' ELSE '' END + 
    CASE WHEN CC_Purchased = 1 THEN ' + CC' ELSE '' END 
     as Cigatype,
    count(distinct id) purchaser_cnt
from temp
group by YYYYMM,
    'IQOS' +
    CASE WHEN CompHnB_Purchased = 1 THEN ' + Comp. HnB' ELSE '' END + 
    CASE WHEN CC_Purchased = 1 THEN ' + CC' ELSE '' END 
;
'''

data3 = pd.read_sql_query(sql=sql3, con=engine)

pivot_current_type = data3.pivot_table(index=['YYYYMM'],
                                       columns='Cigatype',
                                       values='purchaser_cnt',
                                       ).sort_values(['YYYYMM'])

print(pivot_current_type)

pivot_current_type.to_clipboard()

Cigatype    IQOS  IQOS + CC  IQOS + Comp. HnB  IQOS + Comp. HnB + CC
YYYYMM                                                              
202302    2007.0     2035.0             258.0                  376.0
202303    3957.0     3862.0             486.0                  592.0
202304    3511.0     3598.0             436.0                  549.0
202305    3312.0     3991.0             498.0                  643.0
202306    2883.0     3422.0             374.0                  498.0
202307    2591.0     3360.0             314.0                  490.0
202308    2323.0     3095.0             307.0                  477.0
202309    2470.0     3485.0             405.0                  556.0
202310    2587.0     3567.0             365.0                  561.0
202311    2276.0     3092.0             334.0                  497.0
202312    2238.0     3110.0             310.0                  502.0
202401    2170.0     3091.0             337.0                  535.0
202402    2165.0     3285.0       

In [23]:
# - Terea_user_Current_type_M1 (NEO/MIIX/FIIT 전용)

sql3 = ''' 
with temp as (
select  
	t.YYYYMM, 
	t.id,
	max(case when b.cigatype='HnB' and b.company = 'PMK' then 1 else 0 end) IQOS_Purchased,
	max(case when b.cigatype='CC' then 1 else 0 end) CC_Purchased,
	max(case when b.cigatype='HnB' and b.company != 'PMK' then 1 else 0 end) CompHnB_Purchased
from  cu.agg_CU_TEREA_Total_Sourcing t
	join cu.Fct_BGFR_PMI_Monthly a on t.id = a.id 
		and a.YYYYMM = t.YYYYMM
	join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and CIGADEVICE =  'CIGARETTES' AND  b.cigatype != 'CSV' 
	join cu.dim_Regional_area c on t.SIDO_nm = c.sido_nm
group BY 	    	
	t.YYYYMM, 
	t.id
)
select YYYYMM,
    CASE WHEN CompHnB_Purchased = 1 THEN 'Comp. HnB' ELSE '' END + 
    CASE WHEN CC_Purchased = 1 THEN ' + CC' ELSE '' END +
    CASE WHEN IQOS_Purchased = 1 THEN ' + IQOS' ELSE '' END 
     as Cigatype,
    count(*) purchaser_cnt
from temp
group by YYYYMM,
    CASE WHEN CompHnB_Purchased = 1 THEN 'Comp. HnB' ELSE '' END + 
    CASE WHEN CC_Purchased = 1 THEN ' + CC' ELSE '' END +
    CASE WHEN IQOS_Purchased = 1 THEN ' + IQOS' ELSE '' END 
'''

data3 = pd.read_sql_query(sql=sql3, con=engine)

pivot_current_type = data3.pivot_table(index=['YYYYMM'],
                                       columns='Cigatype',
                                       values='purchaser_cnt',
                                       ).sort_values(['YYYYMM'])

print(pivot_current_type)


Cigatype  Comp. HnB  Comp. HnB + CC  Comp. HnB + CC + IQOS  Comp. HnB + IQOS
YYYYMM                                                                      
202302        613.0          1954.0                  119.0             120.0
202303        803.0          1954.0                  111.0             119.0
202304        711.0          1695.0                  136.0             100.0
202305        820.0          1801.0                  139.0             131.0
202306        647.0          1349.0                  112.0             111.0
202307        562.0          1357.0                  112.0              87.0
202308        492.0          1351.0                   89.0              95.0
202309        451.0          1126.0                   83.0              64.0
202310        645.0          1591.0                  116.0              98.0
202311        574.0          1289.0                   86.0              97.0
202312        559.0          1401.0                  121.0              97.0

In [25]:
pivot_current_type.to_clipboard()

# CC 인경우 Brand Family 별로 구매 조회

In [12]:
#-- CC인 경우 TEREA Sourcing 기반 Company 조회

sql_cc = ''' 
select 
	t.YYYYMM, b.company,
    count(distinct t.id ) n
from cu.agg_CU_TEREA_Total_Sourcing  t
	join cu.Fct_BGFR_PMI_Monthly a on a.id = t.id 
		and a.YYYYMM BETWEEN CONVERT(NVARCHAR(6), DATEADD(MONTH, -3, t.YYYYMM+'01'), 112)
				 	     AND CONVERT(NVARCHAR(6), DATEADD(MONTH, -1, t.YYYYMM+'01'), 112)	
	join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and CIGADEVICE =  'CIGARETTES' AND b.cigatype = 'CC'
where 1=1
group BY t.YYYYMM , b.company
order by t.YYYYMM , company 
'''

cc_df = pd.read_sql_query(sql=sql_cc, con=engine)

pivot_cc = cc_df.pivot_table(index=['YYYYMM'],
                  columns='company',
                  values='n',
                  aggfunc='sum'
                   ).sort_values('YYYYMM')



pivot_cc
pivot_cc.to_clipboard()

In [13]:
pivot_cc.to_clipboard()

In [4]:
#-- CC인 경우 TEREA Sourcing 기반 TMO / Brand Family 조회

sql_cc = ''' 
select 
	t.YYYYMM, ProductFamilyCode, b.company,
    count(distinct t.id ) n
from cu.agg_CU_TEREA_Total_Sourcing  t
	join cu.Fct_BGFR_PMI_Monthly a on a.id = t.id 
		and a.YYYYMM BETWEEN CONVERT(NVARCHAR(6), DATEADD(MONTH, -3, t.YYYYMM+'01'), 112)
				 	     AND CONVERT(NVARCHAR(6), DATEADD(MONTH, -1, t.YYYYMM+'01'), 112)	
	join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and CIGADEVICE =  'CIGARETTES' AND b.cigatype = 'CC'
where 1=1
group BY t.YYYYMM, ProductFamilyCode , b.company
order by company,  ProductFamilyCode 
'''

cc_df = pd.read_sql_query(sql=sql_cc, con=engine)

pivot_cc = cc_df.pivot_table(index=['YYYYMM'],
                  columns='company',
                  values='n',
                  aggfunc='sum'
                   ).sort_values('YYYYMM')

pivot2_cc = cc_df.pivot_table(index=['YYYYMM'],
                  columns= [ 'company', 'ProductFamilyCode'],
                  values='n',
                  aggfunc='sum'
                  ).sort_values(['YYYYMM'])

total_cc = pd.concat([pivot_cc, pivot2_cc], axis=1 )

total_cc
total_cc.to_clipboard()

In [5]:
total_cc.to_clipboard()

In [15]:
#--  Total CC Company

sql_cc = ''' 

with temp as (
	select
		t.YYYYMM, 
		t.id, 
	    CASE 
	        WHEN SUM(CASE WHEN b.cigatype = 'CC' and a.YYYYMM = t.YYYYMM THEN 1 ELSE 0 END) > 0 
	         AND SUM(CASE WHEN b.cigatype = 'HnB' and a.YYYYMM = t.YYYYMM THEN 1 ELSE 0 END) > 0 
	        THEN 'Mixed' 
	        ELSE MAX(b.cigatype)  -- CC 또는 HnB가 없을 경우 가장 큰 값을 사용
	    END AS cigatype
    from cu.user_3month_list t
		join cu.Fct_BGFR_PMI_Monthly a on a.id = t.id and a.YYYYMM = t.YYYYMM
		join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and CIGADEVICE =  'CIGARETTES' AND b.cigatype != 'CSV'
	where t.YYYYMM >= '202302'
	group BY t.YYYYMM, t.id
	having sum(pack_qty) > 1
)
select 
	t.YYYYMM, b.company,
    count(distinct t.id ) n
from temp t
	join cu.Fct_BGFR_PMI_Monthly  a on a.id = t.id and a.YYYYMM = t.YYYYMM
	join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and CIGADEVICE = 'CIGARETTES' AND b.cigatype = 'CC'
where 1=1
group BY t.YYYYMM , b.company
;
'''

cc_df = pd.read_sql_query(sql=sql_cc, con=engine)

pivot_cc = cc_df.pivot_table(index=['YYYYMM'],
                  columns='company',
                  values='n',
                  aggfunc='sum'
                   ).sort_values('YYYYMM')

pivot_cc
pivot_cc.to_clipboard()

In [6]:
#--  Total CC TMO / Brand Family 조회

sql_cc = ''' 
with temp as (
	select
		t.YYYYMM, 
		t.id, 
	    CASE 
	        WHEN SUM(CASE WHEN b.cigatype = 'CC' and a.YYYYMM = t.YYYYMM THEN 1 ELSE 0 END) > 0 
	         AND SUM(CASE WHEN b.cigatype = 'HnB' and a.YYYYMM = t.YYYYMM THEN 1 ELSE 0 END) > 0 
	        THEN 'Mixed' 
	        ELSE MAX(b.cigatype)  -- CC 또는 HnB가 없을 경우 가장 큰 값을 사용
	    END AS cigatype
    from cu.user_3month_list t
		join cu.Fct_BGFR_PMI_Monthly a on a.id = t.id and a.YYYYMM = t.YYYYMM
		join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and CIGADEVICE =  'CIGARETTES' AND b.cigatype != 'CSV'
	where t.YYYYMM >= '202302'
	group BY t.YYYYMM, t.id
	having sum(pack_qty) > 1
)
select 
	t.YYYYMM, ProductFamilyCode, b.company,
    count(distinct t.id ) n
from temp t
	join cu.Fct_BGFR_PMI_Monthly  a on a.id = t.id and a.YYYYMM = t.YYYYMM
	join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and CIGADEVICE = 'CIGARETTES' AND b.cigatype = 'CC'
where 1=1
group BY t.YYYYMM, ProductFamilyCode , b.company
;
'''

cc_df = pd.read_sql_query(sql=sql_cc, con=engine)

pivot_cc = cc_df.pivot_table(index=['YYYYMM'],
                  columns='company',
                  values='n',
                  aggfunc='sum'
                   ).sort_values('YYYYMM')


pivot2_cc = cc_df.pivot_table(index=['YYYYMM'],
                  columns= [ 'company', 'ProductFamilyCode'],
                  values='n',
                  aggfunc='sum'
                  ).sort_values(['YYYYMM'])

total_cc = pd.concat([pivot_cc, pivot2_cc], axis=1 )

total_cc
total_cc.to_clipboard()

# 연습

In [14]:
# 필터링된 데이터 생성
filtered_data = data[data['ProductFamilyCode'] == 'IQOS']
 
# 피벗 테이블 생성
pivot_table4 = filtered_data.pivot_table(index=['YYYYMM', 'id', 'gender', 'age'],
                                         columns=['engname'], values=['qty'],
                                         aggfunc='sum')
 
# HnB_taste 컬럼 생성 및 피벗 테이블 생성
result = data[data['productSubFamilyCode'].isin(['AIIM', 'FIIT', 'HEETS', 'MIIX', 'NEO', 'NEOSTICKS', 'TEREA'])]
data['HnB_taste'] = result['productSubFamilyCode'] + ' ' + data['New_FLAVORSEG']

pivot_table3 = data.pivot_table(index=['YYYYMM', 'id', 'gender', 'age'],
                                columns=['HnB_taste'], values=['qty'],
                                aggfunc='sum')
 
# CC_taste 컬럼 생성 및 피벗 테이블 생성
data['CC_taste'] = data['cigatype'] + ' ' + data['New_FLAVORSEG']

pivot_table2 = data.pivot_table(index=['YYYYMM', 'id', 'gender', 'age'],
                                columns=['CC_taste'], values=['qty'],
                                aggfunc='sum')
 
# 기본 피벗 테이블 생성
pivot_table1 = data.pivot_table(index=['YYYYMM', 'id', 'gender', 'age'],
                                columns=['company'], values=['qty'],
                                aggfunc='sum')

# 피벗 테이블들을 하나의 데이터프레임으로 병합
concatenated_pivot = pd.concat([ pivot_table1, pivot_table2, pivot_table3, pivot_table4], axis=1)

# 결과 출력
# final = pd.merge(result, concatenated_pivot, how='outer')
concatenated_pivot

qty  \
                                                                       BAT   
YYYYMM id                                                 gender age         
202211 003DE9124299CC396950FEF717EFE5020EFCB079E8DECB8... 남      40대  32.0   
       007BE5AEC24991C378435D650C80E7C168971700CB8EDFD... 남      40대   NaN   
       00851229FF4A0026F2682594CEDABB0AE1B73FF85E6CDED... 남      50대   NaN   
       00FA9CFB5FFA3F6E3C00DA693D6862019C5A42D6A6D0B35... 남      30대   NaN   
       01117EF5EFBB1B6D53108D3EEBF53FAFBECFA132F880CFB... 남      30대   NaN   
...                                                                    ...   
       FF83E54787EB82C6293ADA53E369F68215B39162BD33E3E... 남      50대   NaN   
       FFC50D9BE8D22E040EDAE5B54B00FA3856C2240DE21BE4C... 남      20대   NaN   
       FFC7C4D03CF065384860A5176E832123A07E70B6E185396... 여      50대   NaN   
       FFD0E8F349A14622CD91B697DA35E0E9D4FFAF7D9D4C4EA... 남      40대   NaN   
       FFEA5E5C9C441901DFBBAA534D9EF782B50A7B88D52B6BA... 여      50대   1.0   

                                                                               \
                                                                     JTI  KTG   
YYYYMM id                                                 gender age            
202211 003DE9124299CC396950FEF717EFE5020EFCB079E8DECB8... 남      40대 NaN  NaN   
       007BE5AEC24991C378435D650C80E7C168971700CB8EDFD... 남      40대 NaN  NaN   
       00851229FF4A0026F2682594CEDABB0AE1B73FF85E6CDED... 남      50대 NaN  NaN   
       00FA9CFB5FFA3F6E3C00DA693D6862019C5A42D6A6D0B35... 남      30대 NaN  9.0   
       01117EF5EFBB1B6D53108D3EEBF53FAFBECFA132F880CFB... 남      30대 NaN  2.0   
...                                                                   ..  ...   
       FF83E54787EB82C6293ADA53E369F68215B39162BD33E3E... 남      50대 NaN  NaN   
       FFC50D9BE8D22E040EDAE5B54B00FA3856C2240DE21BE4C... 남      20대 NaN  1.0   
       FFC7C4D03CF065384860A5176E832123A07E70B6E185396... 여      50대 NaN  4.0   
       FFD0E8F349A14622CD91B697DA35E0E9D4FFAF7D9D4C4EA... 남      40대 NaN  2.0   
       FFEA5E5C9C441901DFBBAA534D9EF782B50A7B88D52B6BA... 여      50대 NaN  9.0   

                                                                            \
                                                                       PMK   
YYYYMM id                                                 gender age         
202211 003DE9124299CC396950FEF717EFE5020EFCB079E8DECB8... 남      40대   NaN   
       007BE5AEC24991C378435D650C80E7C168971700CB8EDFD... 남      40대   4.0   
       00851229FF4A0026F2682594CEDABB0AE1B73FF85E6CDED... 남      50대   7.0   
       00FA9CFB5FFA3F6E3C00DA693D6862019C5A42D6A6D0B35... 남      30대   NaN   
       01117EF5EFBB1B6D53108D3EEBF53FAFBECFA132F880CFB... 남      30대  13.0   
...                                                                    ...   
       FF83E54787EB82C6293ADA53E369F68215B39162BD33E3E... 남      50대  10.0   
       FFC50D9BE8D22E040EDAE5B54B00FA3856C2240DE21BE4C... 남      20대   NaN   
       FFC7C4D03CF065384860A5176E832123A07E70B6E185396... 여      50대   NaN   
       FFD0E8F349A14622CD91B697DA35E0E9D4FFAF7D9D4C4EA... 남      40대   NaN   
       FFEA5E5C9C441901DFBBAA534D9EF782B50A7B88D52B6BA... 여      50대   1.0   

                                                                               \
                                                                     CC Fresh   
YYYYMM id                                                 gender age            
202211 003DE9124299CC396950FEF717EFE5020EFCB079E8DECB8... 남      40대      NaN   
       007BE5AEC24991C378435D650C80E7C168971700CB8EDFD... 남      40대      NaN   
       00851229FF4A0026F2682594CEDABB0AE1B73FF85E6CDED... 남      50대      NaN   
       00FA9CFB5FFA3F6E3C00DA693D6862019C5A42D6A6D0B35... 남      30대      NaN   
       01117EF5EFBB1B6D53108D3EEBF53FAFBECFA132F880CFB... 남      30대      NaN   
...                                                                       ...   
       FF83E54787EB82

In [16]:
total_id_count = data.groupby('YYYYMM')['id'].count().reset_index(name='total_id_count')

gender_age_count = data.groupby(['YYYYMM', 'gender', 'age']).size().reset_index(name='count')

gender_pivot = gender_age_count.pivot_table(
    index ='YYYYMM',
    columns= ['gender', 'age'],
    values='count'
)

res = total_id_count.set_index('YYYYMM').join(gender_pivot)

res

MergeError: Not allowed to merge between different levels. (1 levels on the left, 2 on the right)

In [42]:
# -- Gr Region 집계 user_current_type_M1

gr_sql1= ''' 
with temp as (
select  
	t.YYYYMM, 
	gr_cd,
	t.id,
	max(case when b.cigatype='HnB' and b.company = 'PMK' then 1 else 0 end) IQOS_Purchased,
	max(case when b.cigatype='CC' then 1 else 0 end) CC_Purchased,
	max(case when b.cigatype='HnB' and b.company != 'PMK' then 1 else 0 end) CompHnB_Purchased
from  cu.agg_CU_TEREA_Sourcing t
	join cu.Fct_BGFR_PMI_Monthly a on t.id = a.id 
		and a.YYYYMM = t.YYYYMM
	join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and CIGADEVICE =  'CIGARETTES' AND  b.cigatype != 'CSV' 
	join cu.dim_Regional_area c on t.SIDO_nm = c.sido_nm
group BY 	    	
	t.YYYYMM, 
	gr_cd,
	t.id
)
select YYYYMM, gr_cd,
    'IQOS' +
    CASE WHEN CompHnB_Purchased = 1 THEN ' + Comp. HnB' ELSE '' END + 
    CASE WHEN CC_Purchased = 1 THEN ' + CC' ELSE '' END 
     as Cigatype,
    count(*) purchaser_cnt
from temp
group by YYYYMM, gr_cd,
    'IQOS' +
    CASE WHEN CompHnB_Purchased = 1 THEN ' + Comp. HnB' ELSE '' END + 
    CASE WHEN CC_Purchased = 1 THEN ' + CC' ELSE '' END 
;
'''


gr_data = pd.read_sql_query(sql=gr_sql1, con=engine)

In [49]:
pivot_current_type = gr_data.pivot_table(index=['YYYYMM','gr_cd'],
                                       columns='Cigatype',
                                       values='purchaser_cnt',
                                       ).sort_values(['YYYYMM', 'gr_cd'], ascending=[True, False])

print(pivot_current_type)
pivot_current_type.to_clipboard()

Cigatype        IQOS  IQOS + CC  IQOS + Comp. HnB  IQOS + Comp. HnB + CC
YYYYMM gr_cd                                                            
202401 서울      885.0     1274.0             135.0                  190.0
       부산      337.0      430.0              44.0                   90.0
       대전      247.0      373.0              34.0                   68.0
       대구      168.0      283.0              28.0                   47.0
       광주      279.0      347.0              46.0                   74.0
       Other   254.0      385.0              50.0                   66.0
202402 서울      855.0     1280.0             142.0                  184.0
       부산      345.0      493.0              55.0                   68.0
       대전      257.0      405.0              47.0                   64.0
       대구      214.0      261.0              27.0                   45.0
       광주      246.0      406.0              46.0                   59.0
       Other   248.0      441.0              35.0  

In [43]:
gr_sql2 = ''' 
select  
	t.YYYYMM, 
	gr_cd,
	t.id,
	max(case when b.cigatype='HnB' and b.company = 'PMK' then 1 else 0 end) IQOS_Purchased,
	max(case when b.cigatype='CC' then 1 else 0 end) CC_Purchased,
	max(case when b.cigatype='HnB' and b.company != 'PMK' then 1 else 0 end) CompHnB_Purchased
from cu.agg_CU_TEREA_Sourcing t
	join cu.Fct_BGFR_PMI_Monthly a on t.id = a.id 
		and a.YYYYMM BETWEEN CONVERT(NVARCHAR(6), DATEADD(MONTH, -3, t.YYYYMM+'01'), 112)
				 	     AND CONVERT(NVARCHAR(6), DATEADD(MONTH, -1, t.YYYYMM+'01'), 112)	
	join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and CIGADEVICE =  'CIGARETTES' AND  b.cigatype != 'CSV' 
	join cu.dim_Regional_area c on t.SIDO_nm = c.sido_nm
group BY 	    
	t.YYYYMM, 
	gr_cd, 
	t.id
;
'''

gr_data2 = pd.read_sql_query(sql=gr_sql2, con=engine)

In [52]:
# Past Type 
def categorize(row):
    categories = []
    if row['IQOS_Purchased'] == 1:
        categories.append('PMK HnB')
    if row['CC_Purchased'] == 1:
        categories.append('CC')
    if row['CompHnB_Purchased'] == 1:
        categories.append('Comp HnB')
    return ','.join(categories)

gr_data2['retype'] = gr_data2.apply(categorize, axis=1)
result = gr_data2.groupby(['YYYYMM','gr_cd', 'retype']).size().reset_index(name='N')
total = result['N'].sum()

print(result)

     YYYYMM  gr_cd               retype    N
0    202401  Other                   CC  467
1    202401  Other          CC,Comp HnB  102
2    202401  Other             Comp HnB   28
3    202401  Other              PMK HnB   81
4    202401  Other           PMK HnB,CC   54
..      ...    ...                  ...  ...
247  202406     서울             Comp HnB  214
248  202406     서울              PMK HnB  330
249  202406     서울           PMK HnB,CC  222
250  202406     서울  PMK HnB,CC,Comp HnB   32
251  202406     서울     PMK HnB,Comp HnB   25

[252 rows x 4 columns]


In [56]:
pivot_past_type = result.pivot_table( 
                            index=['YYYYMM', 'gr_cd'],
                            columns='retype',
                            values='N'
                            ).sort_values(['YYYYMM', 'gr_cd'], ascending=[True, False])
print(pivot_past_type)
pivot_past_type.to_clipboard()

retype            CC  CC,Comp HnB  Comp HnB  PMK HnB  PMK HnB,CC  \
YYYYMM gr_cd                                                       
202401 서울     1535.0        267.0     141.0    296.0       202.0   
       부산      565.0        105.0      49.0    103.0        64.0   
       대전      484.0         66.0      43.0     75.0        48.0   
       대구      342.0         64.0      40.0     41.0        33.0   
       광주      468.0         84.0      40.0     86.0        58.0   
       Other   467.0        102.0      28.0     81.0        54.0   
202402 서울     1548.0        258.0     164.0    275.0       177.0   
       부산      605.0        102.0      59.0    102.0        74.0   
       대전      504.0        108.0      35.0     61.0        47.0   
       대구      346.0         62.0      38.0     59.0        36.0   
       광주      492.0         89.0      42.0     78.0        46.0   
       Other   518.0        110.0      39.0     74.0        54.0   
202403 서울     1717.0        283.0     172.0    2

In [58]:
# -- TEREA flavorXtar from 202211

gr_sql3 = ''' 
select  
	t.YYYYMM,
	gr_cd,
	concat(FLAVORSEG_type3,' X ', New_TARSEGMENTAT) flavorXtar,
	count(distinct case when b.cigatype ='CC' then t.id end) CC,
	count(distinct case when b.cigatype ='HnB' then t.id end) HnB
from  cu.agg_CU_TEREA_Sourcing t
	join cu.Fct_BGFR_PMI_Monthly a on t.id = a.id 
		and a.YYYYMM BETWEEN CONVERT(NVARCHAR(6), DATEADD(MONTH, -3, t.YYYYMM+'01'), 112)
				 	     AND CONVERT(NVARCHAR(6), DATEADD(MONTH, -1, t.YYYYMM+'01'), 112)	
	join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and b.CIGADEVICE =  'CIGARETTES' AND b.cigatype != 'CSV'  
	join cu.dim_Regional_area c on t.SIDO_nm = c.sido_nm
group BY 
	t.YYYYMM,
	gr_cd,
	concat(FLAVORSEG_type3,' X ', New_TARSEGMENTAT) 
;
'''

gr_data3 = pd.read_sql_query(sql=gr_sql3, con=engine)

In [60]:
pivot_flavor_tar = gr_data3.pivot_table(index=['YYYYMM', 'gr_cd'], 
                                     columns='flavorXtar',
                                     values=['HnB', 'CC' ]
                                    ).sort_values(['YYYYMM', 'gr_cd'], ascending=[True,True])
print(pivot_flavor_tar)
pivot_flavor_tar.to_clipboard()

                   CC                                                        \
flavorXtar   Fresh X  Fresh X 1MG Fresh X Below 1MG Fresh X LTS Fresh X ULT   
YYYYMM gr_cd                                                                  
202401 Other      0.0        35.0               NaN        41.0        15.0   
       광주         0.0        24.0               NaN        26.0         5.0   
       대구         0.0        16.0               NaN        17.0         6.0   
       대전         0.0        20.0               NaN        27.0         8.0   
       부산         0.0        28.0               NaN        24.0         2.0   
       서울         0.0       117.0               NaN       128.0        18.0   
202402 Other      0.0        36.0               NaN        44.0         8.0   
       광주         0.0        31.0               NaN        26.0         3.0   
       대구         0.0        19.0               NaN        12.0         4.0   
       대전         0.0        34.0               NaN 